# Private Predictions with TFE Keras


## Step 3: Private Prediction using TFE Keras - Serving (Client)

Congratulations! After training your model with normal Keras and securing it with TFE Keras, you are ready to request some private predictions. 

In [1]:
import numpy as np
import tensorflow as tf
import tf_encrypted as tfe

from tensorflow.keras.datasets import mnist

2022-11-01 10:03:28.220295: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-01 10:03:28.224157: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-01 10:03:28.224169: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-01 10:03:29.461103: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-01 10:03:29.461127: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to c

## Data

Here, we preprocess our MNIST data. This is identical to how we preprocessed during training.

In [2]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Set up `tfe.serving.QueueClient`

Before querying the model, you just have to connect to it. To do so, you can create a client with `tfe.serving.QueueClient`. This creates a TFE queueing server on the client side that connects to the queueing server set up by `tfe.serving.QueueServer` in **b - Secure Model Serving**. The queue will be responsible for secret sharing the plaintext data before submitting the shares in a prediction request.

Note that we have to use the same configuration as used by the server, including player configuration and protocol.

In [3]:
config = tfe.RemoteConfig.load("./tfe.config")
config.connect_servers()
tfe.set_config(config)
tfe.set_protocol(tfe.protocol.SecureNN())

input_shape = (1, 28, 28, 1)
output_shape = (1, 10)
client = tfe.serving.QueueClient(input_shape=input_shape, output_shape=output_shape)

2022-11-01 10:03:29.853653: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job tfe -> {0 -> localhost:4000, 1 -> localhost:4001, 2 -> localhost:4002}
2022-11-01 10:03:29.853682: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:43173}
2022-11-01 10:03:29.861726: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job tfe -> {0 -> localhost:4000, 1 -> localhost:4001, 2 -> localhost:4002}
2022-11-01 10:03:29.861746: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:43173}
2022-11-01 10:03:29.862315: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:438] Started server with target: grpc://localhost:43173


## Query Model

You are ready to get some private predictions! Calling `client.run` will insert the `image` into the queue created above, secret share the data locally, and submit the shares to the model server in **b - Secure Model Serving**.

In [4]:
# User inputs
num_tests = 3
images, expected_labels = x_test[:num_tests], y_test[:num_tests]

In [5]:
for image, expected_label in zip(images, expected_labels):
    
    res = client.run(image.reshape((1, 28, 28, 1)))

    predicted_label = np.argmax(res)

    print("The image had label {} and was {} classified as {}".format(
        expected_label,
        "correctly" if expected_label == predicted_label else "incorrectly",
        predicted_label))

tf.Tensor(0, shape=(), dtype=int32)
The image had label 7 and was correctly classified as 7
tf.Tensor(0, shape=(), dtype=int32)
The image had label 2 and was correctly classified as 2
tf.Tensor(0, shape=(), dtype=int32)
The image had label 1 and was correctly classified as 1


This is great. You are able to classify these three images correctly! But what's special about these predictions is that you haven't revealed any private information to get this service. The model host never saw your input data or your predictions, and you never downloaded the model. You were able to get private predictions on encrypted data with an encrypted model!

Before we rush off to apply this in our own apps, let's quickly go back to **b - Secure Model Serving** to clean up our served model.